In [1]:
import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from srgan2 import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# !TF_ENABLE_GPU_GARBAGE_COLLECTION=false

In [3]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [4]:
input_dir = 'data/train/low/'
tgt_dir = 'data/train/high/'

input_dir_val = 'data/val/low/'
tgt_dir_val = 'data/val/high/'


test_file = 'UBMk30rjy0o_17675_1.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_path = 'data/out/' + test_file
test_test_path = 'data/test/' + test_file

vgg_path = 'vgg16_notop.hdf5'

input_shape = (270, 480, 3)
tgt_shape = (540, 960, 3)

batch_size = 2

In [5]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)
gen_valid = ImageDataGenerator(input_dir_val, tgt_dir_val, batch_size)

/home/keras/srgan2.py:141: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("co...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')
/home/keras/srgan2.py:178: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="discriminator", inputs=Tensor("in..., outputs=Tensor("de...)`
  self.shared_discriminator = Network(input=layers[0], output=layers[-1], name='discriminator')
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
# test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
# test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

# test_img_sr = srgan.generator.predict(test_img_low)
# test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
# cv2.imwrite(test_test_path, test_img_sr_int)

In [7]:
for e in range(10001):
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 100 == 0:
        print('epoch', e)
        srgan.valid(X_low, X_high)
        
        test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
        test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
        test_out_path = 'data/out/'  + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:0.16383233666419983, d_loss_fake:1.8904387950897217, vgg_loss:19.953641891479492
epoch 100
d_loss_real:0.7454285025596619, d_loss_fake:0.8345519304275513, vgg_loss:6.087520122528076
epoch 200
d_loss_real:0.6881418228149414, d_loss_fake:0.7408909797668457, vgg_loss:5.624998092651367
epoch 300
d_loss_real:0.7396354675292969, d_loss_fake:0.6543416976928711, vgg_loss:2.503845691680908
epoch 400
d_loss_real:0.7505342960357666, d_loss_fake:0.6520079374313354, vgg_loss:2.213921070098877
epoch 500
d_loss_real:0.6858454346656799, d_loss_fake:0.7184427976608276, vgg_loss:2.9925789833068848
epoch 600
d_loss_real:0.6818228960037231, d_loss_fake:0.6586761474609375, vgg_loss:1.7524034976959229
epoch 700
d_loss_real:0.6743528842926025, d_loss_fake:0.6522160172462463, vgg_loss:1.6317652463912964
epoch 800
d_loss_real:0.702390730381012, d_loss_fake:0.6555118560791016, vgg_loss:2.979038953781128
epoch 900
d_loss_real:0.696840763092041, d_loss_fake:0.7049782276153564, vgg_loss:3.71966

KeyboardInterrupt: 

In [8]:
srgan.generator.save('20200511_generator.h5', include_optimizer=False)

In [12]:
from keras.models import Model, load_model
from keras.engine.network import Network
from keras.layers import Input

# pre_model = load_model('20200510_generator.h5')
pre_model = srgan.generator
pre_net = Network(inputs=pre_model.input, outputs=pre_model.get_layer(pre_model.layers[-1].name).output, name='learned')

input_layer = Input(shape = (540, 960, 3))
sr_out = pre_net(input_layer)
sr_generator = Model(input_layer, sr_out)


In [14]:
img_list = []
for i in range(4):
    test_file = f'UBMk30rjy0o_17675_{i}.jpg'
    path = input_dir_val + test_file
    test_img_low = ((cv2.imread(path) - 127.5) / 127.5)
    img_list.append(test_img_low)

h0 = np.vstack([img_list[0], img_list[1]])
h1 = np.vstack([img_list[2], img_list[3]])
img = np.hstack([h0, h1])
img_gen = sr_generator.predict(img.reshape((1, 540, 960, 3)))

img_int = ((img_gen * 127.5) + 127.5).astype('u1').reshape((1080, 1920, 3))
cv2.imwrite('20200511_out_premerged.jpg', img_int)

True